In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt

In [2]:
IMAGE_SIZE = 256
BATCH_SIZE = 32
PLANT_LIST = ["Pepper","Potato","Tomato"]
data = {}

In [3]:
def load_dataset():
    for plant in PLANT_LIST:
        directory = "PlantVillage/" + plant 
        
        data[plant] = tf.keras.preprocessing.image_dataset_from_directory(
            directory, 
            shuffle = True,
            image_size = (IMAGE_SIZE,IMAGE_SIZE),
            batch_size = BATCH_SIZE
        )
        
    return 0

In [4]:
load_dataset()

Found 2475 files belonging to 2 classes.
Found 2152 files belonging to 3 classes.
Found 16011 files belonging to 10 classes.


0

In [5]:
class_names = {}
for plant in PLANT_LIST:
    class_names[plant] = data[plant].class_names
print(class_names)

{'Pepper': ['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy'], 'Potato': ['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy'], 'Tomato': ['Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato__Tomato_mosaic_virus', 'Tomato_healthy']}


In [6]:
def dataset_partition(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed = 12)
    
    train_sz = int(len(ds) * train_split)
    val_sz = int(len(ds) * val_split)
    
    train_ds = ds.take(train_sz)
    val_ds = ds.skip(train_sz).take(val_sz)
    test_ds = ds.skip(train_sz).skip(val_sz)
    
    return train_ds, val_ds, test_ds

In [7]:
train_ds = {}
val_ds = {}
test_ds = {}

for plant in PLANT_LIST:
    train_ds[plant], val_ds[plant], test_ds[plant] = dataset_partition(data[plant])

In [8]:
for plant in PLANT_LIST:
    train_ds[plant] = train_ds[plant].cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
    val_ds[plant] = val_ds[plant].cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
    test_ds[plant] = test_ds[plant].cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)